In [3]:
#Modern Portfolio Theory
#also known as mean-variance analyse, is a mathematical process which allows the user to maximize returns for 
#a given risk level


#Efficient frontier 
#is a graph with ‘returns’ on the Y-axis and ‘volatility’ on the X-axis.
#It shows the set of optimal portfolios that offer the highest expected return for 
#a given risk level or the lowest risk for a given level of expected return.

#Portfolios that lie outside the efficient frontier are sub-optimal because they do not provide either enough return 
#for the level of risk or have a higher risk for the defined rate of return.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn
import warnings
warnings.filterwarnings("ignore")

# yahoo finance is used to fetch data
import yfinance as yf

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Creating  a linear regression model
from sklearn.linear_model import LinearRegression

from pmdarima import auto_arima

In [25]:
fb = yf.download('FB','2018-01-01','2021-10-11')

[*********************100%***********************]  1 of 1 completed


In [26]:
tesla=yf.download('TSLA','2018-01-01','2021-10-11')

[*********************100%***********************]  1 of 1 completed


In [27]:
fb[['Adj Close']].head()


,Adj Close
Date,
2018-01-02,181.419998
2018-01-03,184.669998
2018-01-04,184.330002
2018-01-05,186.850006
2018-01-08,188.279999


In [28]:
df= pd.DataFrame()

In [29]:
df['FB']=fb['Adj Close']

In [30]:
df['TESL']=tesla['Adj Close']

In [31]:
df.head()

,FB,TESL
Date,,
2018-01-02,181.419998,64.106003
2018-01-03,184.669998,63.450001
2018-01-04,184.330002,62.924000
2018-01-05,186.850006,63.316002
2018-01-08,188.279999,67.281998


In [32]:
df.isna().sum()

FB      0
TESL    0
dtype: int64

In [33]:
df.shape

(950, 2)

In [42]:
#Calculate percentage change in stock prices
tesla_df=df['TESL'].pct_change().apply(lambda x:np.log(1+x))

In [43]:
tesla_df.head()

Date
2018-01-02         NaN
2018-01-03   -0.010286
2018-01-04   -0.008325
2018-01-05    0.006210
2018-01-08    0.060755
Name: TESL, dtype: float64

In [44]:
#The variance in prices of stocks of Tesla are an important indicator
#of how volatile this investment will be (how returns can fluctuate).

In [45]:
var_tesla=tesla_df.var()

In [46]:
var_tesla

0.0016767258836135228

In [48]:
#Calculate percentage change in stock prices
fb_df=df['FB'].pct_change().apply(lambda x:np.log(1+x))

In [49]:
fb_df.head()

Date
2018-01-02         NaN
2018-01-03    0.017756
2018-01-04   -0.001843
2018-01-05    0.013579
2018-01-08    0.007624
Name: FB, dtype: float64

In [50]:
# Variance
var_fb = fb_df.var()
var_fb

0.0005333697349799156

In [ ]:
#Volatility
#is measured as the standard deviation of a company’s stock.

In [51]:
tesla_vol=np.sqrt(var_tesla*250)
fb_vol = np.sqrt(var_fb * 250)

In [55]:
test=pd.DataFrame()

In [56]:
test['Value']=0

In [57]:
test['Value']=(tesla_vol,fb_vol)

In [58]:
test.head()

,Value
0,0.647442
1,0.365161


In [61]:
test.columns=['Volatile']

In [ ]:
#Covariance measures the directional relationship between the returns on two assets.

In [63]:
test1=df.pct_change().apply(lambda x:np.log(1+x))

In [64]:
test1.head()

,FB,TESL
Date,,
2018-01-02,NaN,NaN
2018-01-03,0.017756,-0.010286
2018-01-04,-0.001843,-0.008325
2018-01-05,0.013579,0.006210
2018-01-08,0.007624,0.060755


In [66]:
test1['TESL'].cov(test1['FB'])

0.00031887824254016686

In [ ]:
#Correlation, in the finance and investment industries, 
#is a statistic that measures the degree to which two securities move in relation to each other. 
#Correlations are used in advanced portfolio management, computed as the correlation coefficient, 
#which has a value that must fall between -1.0 and +1.0.

In [67]:
test1['TESL'].corr(test1['FB'])

0.337193868668146

In [ ]:
#Expected returns of an asset are simply the mean of percentage change in its stock prices. 
#So, the value of expected return we obtain here are daily expected returns.

#For an yearly expected return value, you will need to resample the data year-wise, as you will see further.

#For expected returns, you need to define weights for the assets choosen.

In [69]:
test2 = df.pct_change().apply(lambda x: np.log(1+x))
test2.head()

,FB,TESL
Date,,
2018-01-02,NaN,NaN
2018-01-03,0.017756,-0.010286
2018-01-04,-0.001843,-0.008325
2018-01-05,0.013579,0.006210
2018-01-08,0.007624,0.060755


In [ ]:
#Let’s define an array of random weights for the purpose of calculation. 
#These weights will represent the percentage allocation of investments between these two stocks.
#They must add up to 1.

#So, the problem of portfolio optimization is nothing but to find the optimal values of weights that
#maximizes expected returns while minimizing the risk (standard deviation).

In [70]:
w = [0.2, 0.8]
e_r_ind = test2.mean()
e_r_ind

FB      0.000631
TESL    0.002640
dtype: float64

In [71]:
e_r = (e_r_ind*w).sum()
e_r

0.0022384632614242676